# ACIE Training on Google Colab

This notebook provides a step-by-step guide to training the Astronomical Counterfactual Inference Engine (ACIE) on Google Colab's free GPU instances.

## Prerequisites
1. **Upload Code**: Zip your local `ACIE` project folder and upload it to your Google Drive (e.g., in `My Drive/ACIE`).
2. **Upload Data**: Ensure your CSV datasets are also in that folder or a `data/` subfolder.
3. **Runtime**: Go to `Runtime` > `Change runtime type` > Select `T4 GPU` (or better).

In [ ]:
# 1. Check GPU Status
!nvidia-smi

In [ ]:
# 2. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 3. Setup Project Path
import os
import sys

# CHANGE THIS PATH to where you uploaded the ACIE folder
PROJECT_PATH = "/content/drive/My Drive/ACIE"

# If you uploaded a zip, unzip it first (uncomment below)
# !unzip "/content/drive/My Drive/ACIE.zip" -d /content/
# PROJECT_PATH = "/content/ACIE"

if os.path.exists(PROJECT_PATH):
    os.chdir(PROJECT_PATH)
    sys.path.append(PROJECT_PATH)
    print(f"Current working directory: {os.getcwd()}")
else:
    print(f"Error: Path {PROJECT_PATH} does not exist. Please check your Drive structure.")

In [ ]:
# 4. Install Dependencies
print("Installing dependencies...")
!pip install -q pytorch-lightning torchmetrics python-dotenv
!pip install -q "numpy<2.0" pandas scipy networkx
!pip install -q "bcrypt<4.0.0" passlib python-jose[cryptography]  # For API if needed
# Install project in editable mode to ensure local imports work
!pip install -e .

In [ ]:
# 5. verify Installation
import torch
import acie
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"ACIE Package: {acie.__file__}")

In [ ]:
# 6. Run Training
# Configuration
DATA_DIR = "data/"  # Relative to PROJECT_PATH
OUTPUT_DIR = "outputs/colab_run1"
DATASET_SIZE = "10k"
MAX_EPOCHS = 20
BATCH_SIZE = 64

# Construct command
cmd = f"python acie/training/train.py --data_dir {DATA_DIR} --output_dir {OUTPUT_DIR} --dataset_size {DATASET_SIZE} --max_epochs {MAX_EPOCHS} --batch_size {BATCH_SIZE} --gpus 1"

print(f"Running command: {cmd}")
!{cmd}

In [ ]:
# 7. Logs & Checkpoints
# Save outputs back to Drive if not already working there
# !cp -r outputs/ "/content/drive/My Drive/ACIE_Outputs/"

In [ ]:
# 8. Launch TensorBoard (inline)
%load_ext tensorboard
%tensorboard --logdir outputs/colab_run1/logs